In [1]:
import json
import requests
from pprint import pprint
import numpy as np
import pandas as pd
import re
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier, LogisticRegression
import sqlite3
import pickle
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [2]:
### CLASSIFICATION CODE ###
beleidsWoorden = ['bestuur en beleid','facilitering','bevolkingsregister','straatnaamgeving','kadaster','kadastrale','stelposten','begrotingsruimte','integratieuitkeringen','decentralisatieuitkeringen','accountantscontrole','ombudsfunctie','rekenkamer','reiskosten','loonkosten', 'paspoorten', 'rijbewijzen', 'verkiezingen', 'documentaire informatievoorziening ', 'inkoop', 'ozb', 'onroerend zaakbelasting ', 'parkeerbelasting', 'hondenbelasting ', 'precariobeslasting ', 'reclamebelasting ', 'vennootschapsbelasting', 'algemene uitkering ', 'mutaties reserves', 'dividend nutsbedrijven']
veiligheidWoorden = ['veiligheid', 'bibob', 'criminaliteit','criminelen','inbraken','explosieven','radicalisering','doodschouw','buitengewoon opsporingsambtenaar','crisisbeheersing', 'brandweer', 'brandbestrijding', 'rampenbestrijding', 'halt', 'apv', 'boa', 'georganiseerde criminaliteit', 'wet wapen en munitie', 'antidiscriminatiebeleid', 'dierenbescherming ', 'politie', 'leges drank en horeca', 'openbare orde']
onderwijsWoorden = ['onderwijs', 'onderwijshuisvesting', 'school', 'scholen', 'basisschool', 'bewegingsonderwijs', 'leraren', 'vandalismebestrijding', 'schoolgebouwen', 'onderwijsbeleid', 'leerlingzaken', 'leerkrachten', 'volwasseneneducatie', 'peuterspeelzaal', 'peuterspeelzalen', 'leerlingbegeleiding', 'leerlingzorg', 'leerlingenvervoer', 'schooldeelname', 'leerplicht', 'schoolverlaten', 'basisonderwijs']
economieWoorden = ['economische ontwikkeling','economie','economische','winkeliers','ondernemers', 'toerisme', 'landbouw','veeteelt','visserij','promotie', 'toeristen', 'beurzen', 'beurs', 'jaarmarkten', 'forensenbelasting', 'toeristenbelasting', 'vermakelijkhedenretributies', 'bedrijvenloket', 'ondernemersloket', 'startende ondernemers', 'straathandel', 'markten', 'biz-bijdrage', 'marktgelden', 'grondexploitatie bedrijventerreinen', 'winkelgebieden', 'bedrijfslocaties']
verkeerWoorden = ['verkeer', 'verkeersbeleid', 'verkeersmaatregelen', 'verkeersveiligheid', 'wegen', 'pleinen', 'fietspad', 'voetpad', 'straten', 'civieltechnische kunstwerken', 'straatverlichting', 'gladheidbestrijding', 'sneeuwruimen', 'strooien', 'straatreiniging', 'zwerfafval', 'parkeerbeleid', 'parkeermeters', 'parkeervoorzieningen ', 'jachthaven ', 'bruggelden', 'passantenhaven', 'liggelden', 'havengelden', 'baggerwerkzaamheden ', 'zeehavens', 'binnenhavens ', 'doorgaande waterwegen', 'waterkering ', 'afwatering', 'openbaar vervoer', 'bus', 'tram', 'metro', 'veerdiensten', 'taxivervoer', 'busstation', 'metrostation', 'multimodaal knooppunt']
cultuurWoorden = ['sport en cultuur', 'sportbeleid', 'topsport', 'sportbeoefening', 'recreatie', 'sport', 'cultuur', 'recreatieve', 'sportaccommodaties','sportverenigingen', 'sportvelden', 'zwembad', 'schaatshal', 'trapveldje', 'voetbalveld', 'voetbal', 'hockey', 'volleybal', 'handbal', 'muziek', 'dans', 'toneel', 'kunst', 'cultuurparticipatie', 'cultuurpresentatie', 'cultuurproductie', 'cultuuruitingen', 'kunstenaars', 'kunstwerken', 'cultuureducatie', 'herdenking', 'musea', 'museum', 'expositie', 'archeologie','archeologisch', 'heemkunde', 'historische archieven', 'bibliotheken', 'bibliotheek','stadsgezicht','monumenten','monumentenwet','lokale pers', 'lokale omroep', 'lokale informatievoorziening', 'natuurbescherming', 'onderhoud van bos', 'vijvers', 'vijver', 'openbaar groen', 'betuining', 'hobbyclub', 'volkstuinvereniging', 'speelvoorziening', 'recreatievoorziening']
gezondheidsWoorden = ['volksgezondheid', 'milieu', 'gezondheidssituatie', 'gezondheid', 'gezondheidsbevordering', 'infectieziekten', 'vaccinaties', 'vaccinatie','prenatale', 'psychosociale hulp', 'centrum jeugd en gezin', 'ziekenvervoer', 'ambulance', 'riolering', 'oppervlaktewater', 'rioolwaterzuivering', 'rioolheffing', 'grondwater', 'afvalwater', 'hemelwater', 'afval', 'vuilophaal', 'reinigingsrechten', 'vuilstort', 'afvalstoffenheffing', 'milieubeheer', 'geluidhinder', 'bodemsanering', 'rud', 'regionale uitvoeringsdiensten', 'ongediertebestrijding', 'recycling', 'begraafplaatsen', 'begraafplaats','crematoria', 'grafrechten', 'luchtkwaliteit', 'milieubeleid', 'milieukwaliteit']
sociaalWoorden = ['sociaal domein', 'burgerparticipatie', 'amw', 'algemeen maatschappelijk werk', 'burgerinitiatieven', 'eenzaamheidsbestrijding', 'collectief aanvullend vervoer', 'kinderopvang', 'noodopvang vluchtelingen', 'vreemdelingen', 'wijkteams', 'inkomensregelingen', 'ioaw', 'ioaz', 'wet inkomensvoorzieningen oudere en gedeeltelijk arbeidsongeschikte werkloze werknemers', 'wet inkomensvoorzieningen oudere en gedeeltelijk arbeidsongeschikte gewezen zelfstandigen', 'bijstandverlening', 'inkomensregeling', 'participatiewet', 'loonkostensubsidies', 'bijstandsverlening', 'bijstand', 'schuldhulpverlening', 'beschut werken', 'work first', 'proefplaatsen', 'participatieplaatsen', 'vrijwilligerswerk', 'detacheringsbanen', 'erkenning van verworven competenties', 'evc', 'loonkostensubsidie', 'jobcoach', 'werkplekaanpassingen', 'werkvoorzieningen', 'doventolk', 'inkomensvrijlating', 'stimuleringspremies', 'onkostenvergoedingen', 'no-riskpolis', 'inburgering', 'cursus nederlands', 'loonwaardebepaling', 'besluit bijstandsverlening zelfstandigen', 'bbz', 'maatwerkdienstverlening', 'maatwerkvoorzieningen', 'wmo', 'woningaanpassingen', 'gehandicaptenparkeerkaart ', 'eigen bijdrage', 'mantelzorg', 'pgb', 'jeugd-ggz', 'jeugdzorg', 'gescaleerde zorg', 'beschermdwonenvoorzieningen', 'opvangvoorzieningen', 'vrouwenopvang', 'veilig thuis', 'aanpak huiselijk geweld', 'beschermd wonen', 'kinderbeschermingsmaatregelen', 'jeugdreclassering ', 'ouderbijdragen']
wonenWoorden = ['volkshuisvesting en ruimtelijke ordening', 'ruimtelijke ordening', 'bgt', 'basisregistratie grootschalige topografie', 'bestemmingsplan', 'bestemmingsplannen', 'grondbeleid', 'cai', 'breedband', 'glasvezel', 'grondexploitatie', 'grondverwerving', 'bouwrijpe gronden', 'bouwgrondcomplexen', 'gebiedsontwikkeling', 'woningvoorraad', 'huisvestingsvoorziening', 'basisregistratie adressen en gebouwen', 'bouwtoezicht', 'bag', 'woningbouw', 'woningverbetering', 'woonruimteverdeling', 'woningsplitsingsvergunning', 'woonvergunning', 'stedelijke vernieuwing', 'woningvoorraad', 'woonomgeving']
lijsten = [beleidsWoorden,veiligheidWoorden,onderwijsWoorden,economieWoorden,verkeerWoorden,sociaalWoorden,wonenWoorden,gezondheidsWoorden,cultuurWoorden]

def classify(jsonDoc,classifiers,tfidfmodel): 
    ### INPUT: jsonDoc, JSON-file vanuit ORI-API zonder aangegeven thema's met daarin mogelijk onderliggende documenten;
    ### classifiers, lijst van getrainde classifiers die worden gebruikt om thema's te vinden in de tekstbestanden (wordt nog veranderd naar FLASK)
    ### tfidfmodel, tfidfmodel gebruikt bij de classifiers (wordt nog veranderd naar FLASK)
    ### OUTPUT: listOfDocs, per onderliggend document van het JSON-FILE een dict met daarin per thema een score;
    jsonDoc = jsonDoc.json()
    ### FLASK tfidfmodel
    ### FLASK classifier
    listOfDocs = getUnderlyingDocs(jsonDoc)
    listOfDocs = prepare(listOfDocs,tfidfmodel)
    listOfDocs = addLabels(listOfDocs,classifiers)
    return listOfDocs

def getUnderlyingDocs(jsonDoc):
    ### INPUT: jsonDoc, JSON-file vanuit ORI-API zonder aangegeven thema's met daarin mogelijk onderliggende documenten;
    ### OUTPUT: listOfDocs, per onderliggend document een dict met een ID en een string van de inhoud van het document;
    listOfDocs = []
    docID = jsonDoc["id"]
    text = jsonDoc["name"]
    if "description" in jsonDoc.keys():
        text += " " + jsonDoc["description"]
    listOfDocs.append({"id":docID, "text":text}) #Dit is niet een onderliggend document, maar de titel en de omschrijving van het agendapunt 
    
    if "sources" in jsonDoc.keys(): #loop over de onderliggende documenten
        keys = []
        for doc in jsonDoc["sources"]:
            if "description" in doc.keys() and  docID+"-"+doc["url"] not in keys:
                keys.append(docID+"-"+doc["url"])
                listOfDocs.append({"id":docID+"-"+doc["url"], "text":doc["note"]+" "+doc["description"]})        
    return listOfDocs

def load_models(lijsten):
    transformer = TfidfTransformer()
    tfidfModel = pickle.load(open("Models\\tfidf.pickle", "rb"))
    classificationModels = []
    for lijst in lijsten:
        classificationModels.append(pickle.load(open("Models\\"+lijst[0]+ ".pickle", "rb")))
    return classificationModels, tfidfModel

def preprocess(text):
    text = text.lower()
    text = re.sub("<\w*>", '', text)
    text = re.sub("<\w*\s\/>", '', text)
    text = re.sub("^https?:\/\/.*[\r\n]*", '', text)
    text = re.sub('\\\\x\S.', '', text)
    text = re.sub('[^a-z\s]', '', text)
    text = re.sub("\s+", " ", text)
    return text

def prepare(listOfDocs, tfidfModel):
    for doc in listOfDocs:
        doc["text"] = preprocess(doc["text"])
        doc["matrix"] = tfidfModel.transform([doc["text"]])
    return listOfDocs

def addLabels(listOfDocs, classifiers):
    for doc in listOfDocs:
        for clf, lijst in zip(classifiers, lijsten):
            doc[lijst[0]] = clf.predict_proba(doc["matrix"])[0,0]
        del doc["text"]
        del doc["matrix"]
    return listOfDocs
    

In [4]:
### TEST CLASSIFY CODE ###
classifiers,tfidfmodel = load_models(lijsten)
ids = ["9be7ccd1f784da244a44394bd67830b659223870","1502b43aae09ee2836259807bbdbf4cfa65455cc","f87a0a6eed61a41733fb964e56cafc0990129614",
       "985474db88e7eecc75324995bc9d877acd3a9605","c09f1395c428d4cca44fe792ffb4c2ad9ef3c360","0da88aa9e225f142901a12694702e0204b537119"]
for thing in ids:
    jsonDoc = requests.get("http://api.openraadsinformatie.nl/v0/combined_index/events/"+thing)
    pprint(classify(jsonDoc,classifiers,tfidfmodel))

[{'bestuur en beleid': 0.9827681668893659,
  'economische ontwikkeling': 0.9796148205072654,
  'id': '9be7ccd1f784da244a44394bd67830b659223870',
  'onderwijs': 0.9285002545358176,
  'sociaal domein': 0.5389935958534602,
  'sport en cultuur': 0.7351545936692164,
  'veiligheid': 0.9080808680530194,
  'verkeer': 0.8931531866242615,
  'volksgezondheid': 0.845302635370603,
  'volkshuisvesting en ruimtelijke ordening': 0.5312119336075818},
 {'bestuur en beleid': 0.9997016935493253,
  'economische ontwikkeling': 0.9983105482743085,
  'id': '9be7ccd1f784da244a44394bd67830b659223870-https://api.notubiz.nl/document/6829406/1',
  'onderwijs': 0.9938272256209575,
  'sociaal domein': 0.9027845126448077,
  'sport en cultuur': 0.9770640514038649,
  'veiligheid': 0.9995327630147388,
  'verkeer': 0.9998953675938219,
  'volksgezondheid': 0.9996607753597212,
  'volkshuisvesting en ruimtelijke ordening': 0.3389596386925787},
 {'bestuur en beleid': 0.996168480007323,
  'economische ontwikkeling': 0.9816271

[{'bestuur en beleid': 0.9688136302036126,
  'economische ontwikkeling': 0.3789701860795598,
  'id': 'c09f1395c428d4cca44fe792ffb4c2ad9ef3c360',
  'onderwijs': 0.9801209665152072,
  'sociaal domein': 0.9198928978630995,
  'sport en cultuur': 0.9725620121228117,
  'veiligheid': 0.9939815174799597,
  'verkeer': 0.9970605215192049,
  'volksgezondheid': 0.9960689123403857,
  'volkshuisvesting en ruimtelijke ordening': 0.003443953166307856},
 {'bestuur en beleid': 0.9579297309552225,
  'economische ontwikkeling': 0.9977976838351886,
  'id': 'c09f1395c428d4cca44fe792ffb4c2ad9ef3c360-https://api.notubiz.nl/document/6829465/1',
  'onderwijs': 0.9892773202382166,
  'sociaal domein': 0.35666695748741994,
  'sport en cultuur': 0.9968093171455388,
  'veiligheid': 0.9875756106612307,
  'verkeer': 0.9942780347392048,
  'volksgezondheid': 0.9984932952225127,
  'volkshuisvesting en ruimtelijke ordening': 0.35572153719308464}]
[{'bestuur en beleid': 0.619436013840843,
  'economische ontwikkeling': 0.97

In [169]:
### RETRAIN CODE ###
def retrain():
    df = getIDs()
    df["text"] = df.apply(getDocuments, axis=1)
    tfidfModel,x,df = tfidf(df)
    logRegModel = createModel(tfidfModel,x,df)
    return

def getIDs():
    conn = sqlite3.connect('feedback.db')
    cursor = conn.cursor()
    sql_answer = cursor.execute("SELECT * FROM feedback")
    df = pd.DataFrame(sql_answer.fetchall())
    df.columns = ["id","agenda","bestuurEnBeleid","veiligheid","onderwijs","economie","verkeer",
                  "sociaal","volkshuisvesting","gezondheid","cultuur"]
    return df

def getDocuments(row):
    splitID = row["id"].split("-",1)
    pointID = splitID[0]
    JsonDoc = requests.get("http://api.openraadsinformatie.nl/v0/combined_index/events/"+pointID).json()
    if len(splitID) > 1:
        url = splitID[1]
        for source in JsonDoc["sources"]:
            if source["url"] == url:
                text = preprocess(source["note"]+" "+source["description"])
                break        
    else:
        text = preprocess(JsonDoc["name"] +  " " + JsonDoc["description"])
    return text

def tfidf(df):
    stop = []
    for lijst in lijsten:
        for word in lijst:
            stop.extend(word.split())

    df = df.sample(frac=1).reset_index(drop=True)
    tfidfModel = TfidfVectorizer(max_df=0.5,min_df=0.001, stop_words=stop)
    x = tfidfModel.fit_transform(df["text"])
    del df["text"]
    return tfidfModel,x,df

def createModel(tfidfModel,x,df):
    for lijst in ["bestuurEnBeleid","veiligheid","onderwijs","economie","verkeer",
                  "sociaal","volkshuisvesting","gezondheid","cultuur"]:
        y = np.where(df[lijst] > 0, 1, 0)
        train = round(0.8*len(y))
        test = len(y) - train
        
        clf = LogisticRegression(C= 10, penalty= 'l2', solver= 'liblinear',dual=False,max_iter=250).fit(x[:train], y[:train])
        
        pickle.dump(clf, open("Models/new_"+lijst+".pickle", "wb"))
    pickle.dump(tfidfModel, open("Models/new_tfidf.pickle", "wb"))
    return

In [170]:
retrain()

ConnectionError: (MaxRetryError("HTTPConnectionPool(host='api.openraadsinformatie.nl', port=80): Max retries exceeded with url: /v0/combined_index/events/350237384b83316ac4996f2b3993200f4f35379f (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000029DDD9F0CC0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))",), 'occurred at index 0')

In [14]:
### FEEDBACK CODE ###
def feedback(feedbackDict):
    conn = sqlite3.connect('feedback.db')
    cursor = conn.cursor()
    cursor.execute("SELECT " +feedbackDict["category"]+" FROM feedback WHERE id LIKE '" + feedbackDict["id"]+"'")
    if cursor.fetchall():
        cursor.execute("UPDATE feedback SET " +feedbackDict["category"]+"=" +feedbackDict["category"]+"+"
                       +str(feedbackDict["feedback"])+" WHERE id ='" +feedbackDict["id"]+"'")
    else:
        feedbackList = createFeedbackList([feedbackDict["id"],0],feedbackDict["category"],feedbackDict["feedback"])
        cursor.executemany('INSERT INTO feedback VALUES (?,?,?,?,?,?,?,?,?,?,?)', [feedbackList])
    conn.commit() 
    return
    
def createFeedbackList(feedbackList,category,feedback):
    columnnames = ["bestuurEnBeleid","veiligheid","onderwijs","economie","verkeer",
                  "sociaal","wonen","gezondheid","cultuur"]
    for name in columnnames:
        if name is category:
            feedbackList.append(feedback)
        else:
            feedbackList.append(0)
    return feedbackList

In [22]:
### TEST FEEDBACK II
#feedback({"id":"hoi","feedback":4,"category":"veiligheid"})    

conn = sqlite3.connect('feedback.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM feedback WHERE volkshuisvesting = -1")
print(cursor.fetchall())
conn.close()

[('17', 0, 0, 0, 0, 0, 0, 0, -1, 0, 0)]


In [167]:
### TEST FEEDBACK I
conn = sqlite3.connect('feedback.db')
c = conn.cursor()

# Create table
c.execute('''CREATE TABLE feedback
             (id text, agenda int, bestuurEnBeleid int, veiligheid int, onderwijs int, 
             economie int, verkeer int, sociaal int, volkshuisvesting int, gezondheid int, cultuur int)''')

# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()